## imports

In [1]:
import pandas as pd
import gzip

In [42]:
from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()

In [2]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [26]:
!pip install rake_nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
from rake_nltk import Rake

In [62]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import BigramCollocationFinder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 4.9 MB/s 
     |████████████████████████████████| 240 kB 48.3 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=1f9454c34f393ce51d15c2ba5f2db2f50dde6beaf49472ad724ec09730efe055
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [13]:
import stanza
stanza.download('en')


INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


## get data

In [5]:
# code from http://jmcauley.ucsd.edu/data/amazon/

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/content/drive/MyDrive/reviews_Pet_Supplies_5.json.gz')

In [6]:
df.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A14CK12J7C7JRK,1223000893,Consumer in NorCal,"[0, 0]",I purchased the Trilogy with hoping my two cat...,3.0,Nice Distraction for my cats for about 15 minutes,1294790400,"01 12, 2011"
1,A39QHP5WLON5HV,1223000893,Melodee Placial,"[0, 0]",There are usually one or more of my cats watch...,5.0,Entertaining for my cats,1379116800,"09 14, 2013"
2,A2CR37UY3VR7BN,1223000893,Michelle Ashbery,"[0, 0]",I bought the triliogy and have tested out all ...,4.0,Entertaining,1355875200,"12 19, 2012"
3,A2A4COGL9VW2HY,1223000893,Michelle P,"[2, 2]",My female kitty could care less about these vi...,4.0,Happy to have them,1305158400,"05 12, 2011"
4,A2UBQA85NIGLHA,1223000893,"Tim Isenhour ""Timbo""","[6, 7]","If I had gotten just volume two, I would have ...",3.0,You really only need vol 2,1330905600,"03 5, 2012"


In [7]:
reviews = list(df['reviewText'])

In [8]:
len(reviews)

157836

## 1. 

Попробуем посмотреть на то, как выглядят отзывы:

In [18]:
reviews[:3]

["I purchased the Trilogy with hoping my two cats, age 3 and 5 would be interested.  The 3 yr old cat was fascinated for about 15 minutes but when the same pictures came on, she got bored.  The 5 year old watched for about a few minutes but then walked away. It is possible that because we have a wonderful courtyard full of greenery and trees and one of my neighbors has a bird feeder, that there is enough going on outside that they prefer real life versus a taped version.  I will more than likely pass this on to a friend who has cats that don't have as much wildlife to watch as mine do.",
 'There are usually one or more of my cats watching TV and staying out of trouble when this DVD is playing.  They seem to like the mice and birds the most and maybe go a little less stir crazy being inside all the time.',
 "I bought the triliogy and have tested out all the DVDs.  It appears that volume 2 is the most well received of the three and the one I would recommend.  It's funny to watch my cat w

Можно выделить некоторые паттерны: например, люди чаще всего реферируют к товару коллокацией this + NOUN ('this stuff', 'this toothpaste' и т.д.). Кроме того, из-за специфики отзывов возникают коллокации типа purchased the + NOUN, bought the + NOUN.

In [ ]:
# this + noun
# purchased the + noun
# bought the + noun

Можно попробовать выделить сущности с помощью Stanza (которая мне очень нравится и обычно показывает отличные результаты):

In [14]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


Попробуем вывести для некоторых из отзывов:

In [45]:
texts =  reviews[:200]
for t in texts:
  doc = nlp(t)
  for sent in doc.sentences:
    for ent in sent.ents:  # достаем NE
      if ent.type == 'PRODUCT':
        print(ent)

{
  "text": "Blitz",
  "type": "PRODUCT",
  "start_char": 23,
  "end_char": 28
}
{
  "text": "Soft Coated Wheaten Terrier",
  "type": "PRODUCT",
  "start_char": 130,
  "end_char": 157
}
{
  "text": "EverClean",
  "type": "PRODUCT",
  "start_char": 89,
  "end_char": 98
}
{
  "text": "LitterMaid",
  "type": "PRODUCT",
  "start_char": 287,
  "end_char": 297
}
{
  "text": "the Litter Maid",
  "type": "PRODUCT",
  "start_char": 2015,
  "end_char": 2030
}
{
  "text": "Litter Secret",
  "type": "PRODUCT",
  "start_char": 2366,
  "end_char": 2379
}
{
  "text": "the Litter Maid",
  "type": "PRODUCT",
  "start_char": 2674,
  "end_char": 2689
}
{
  "text": "the Litter Maid",
  "type": "PRODUCT",
  "start_char": 3875,
  "end_char": 3890
}
{
  "text": "the Litter Robot",
  "type": "PRODUCT",
  "start_char": 4016,
  "end_char": 4032
}
{
  "text": "Litter Maid",
  "type": "PRODUCT",
  "start_char": 4048,
  "end_char": 4059
}
{
  "text": "the Litter Robot",
  "type": "PRODUCT",
  "start_char": 4150,
 

К сожалению, выделяемые сущности показывают только название товара (или брэнд), что и так можно получить из метаданных. Нам интересны другие референции к товару, например, не 'Littermaid', а 'cat litter box'. На всякий случай добавим результаты в датасет. Поскольку станза будет работать очень долго, возьмем небольшую порцию датасета:

In [51]:
df_small = df.sample(1000)

In [47]:
def stanza_ners(val):
  out = []
  doc = nlp(val)
  for sent in doc.sentences:
    for ent in sent.ents:  # достаем NE
      if ent.type == 'PRODUCT':
        out.append(ent.text)
  return out

In [52]:
df_small['stanza_ners'] = df_small['reviewText'].progress_apply(stanza_ners)

100%|██████████| 1000/1000 [15:47<00:00,  1.05it/s]


In [54]:
df_small.sample()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,keywords,stanza_ners
143775,A1W670KK0YNM74,B0085TVBK6,Donna M. Tierney,"[0, 0]",Using this Litter Genie is saving me both time...,5.0,Excellent Investment,1354924800,"12 8, 2012","[buy small trash bags, trash disposal every da...",[Litter Genie]


Другой подход: выделять ключевые слова, например, с помощью rake-nltk

## 2.

Попробуем реализовать этот подход на одном из отзывов:

In [33]:
rake_nltk_var = Rake()

In [39]:
text = reviews[50]
rake_nltk_var.extract_keywords_from_text(text)
keyword_extracted = rake_nltk_var.get_ranked_phrases()[:10]
print(keyword_extracted)

['would get caked solid rather quickly', 'storage container would get jammed', 'tines would get clumped full', 'littermaid lm500 automated litter box', 'would keep going back', 'tines would decide', 'system sense motion', 'receptacle plastic box', 'random motion thing', 'tines move across']


Хотя не все ключевые слова и фразы относятся к самому продукту (многие содержат отношение автора, что логично), мы все равно получаем достаточную информацию о продукте ('littermaid lm500 automated litter box', 'system sense motion', 'receptacle plastic box'). Чтобы отбирать ключевые слова, относящиеся к продукту непосредственно, мы можем воспользоваться метаданными, в частности названиями продуктов и названиями товаров, часто приобретаемых вместе. К сожалению, файлы с метаданными нужно запрашивать у авторов, однако и без этого отобранные ключевые слова дают достаточно информации о товаре.

In [40]:
def get_keywords(val):
  rake_nltk_var.extract_keywords_from_text(val)
  return rake_nltk_var.get_ranked_phrases()[:10]

In [43]:
df['keywords'] = df['reviewText'].progress_apply(get_keywords)

100%|██████████| 157836/157836 [01:06<00:00, 2362.00it/s]


In [46]:
df.sample()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,keywords
31322,AAXM7JE9QKYTY,B00076NS1U,C. Campedelli,"[2, 2]",I believe when people say that it worked with ...,1.0,did not work with my dog,1344729600,"08 12, 2012","[people say, hyper dog, dog, worked, work, bel..."


##3. 

Поскольку Станза лучше справилась с выделением продуктов, а rank-nltk нужно еще дорабатывать, попробуем искать н-граммы для результатов Станзы

In [57]:
df_small = df_small.reset_index()

In [85]:
left_contexts = []
right_contexts = []
for i in range(len(df_small)):
  text = df_small['reviewText'][i]
  ners = df_small['stanza_ners'][i]
  tokens = word_tokenize(text)
  for t in range(len(tokens)):
    for n in ners:
      nns = n.split(' ')
      if nns[0] == tokens[t] and t != 0:
        left_contexts.append(tokens[t-1].lower() + ' ' + n.lower())
      if nns[-1] == tokens[t] and t != len(tokens):
        right_contexts.append(n.lower() + ' ' + tokens[t+1].lower())


In [86]:
len(left_contexts), len(right_contexts)

(335, 237)

Мы получили н-граммы с сущностями + слово слева и сущность + слово справа:

In [87]:
left_contexts[0], right_contexts[0]

("my labrador retriever's", 'catit because')

## 4.

In [88]:
contexts = left_contexts + right_contexts

In [89]:
len(contexts)

572

In [90]:
contexts = [word_tokenize(x) for x in contexts]

In [91]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder2 = BigramCollocationFinder.from_documents(contexts)

In [92]:
nonorm_finder = BigramCollocationFinder.from_documents(contexts)

In [116]:
finder2.apply_freq_filter(3) # датасет маленький, сильно урезать не стоит

Пробуем разные метрики:

In [117]:
finder2.score_ngrams(bigram_measures.likelihood_ratio)

[(('purrfectly', 'chicken'), 277.65608115109995),
 (('frenzy', 'refill'), 257.3305140751368),
 (('ratatouille', 'frenzy'), 257.3305140751368),
 (('ubiquinol', 'coq10'), 191.70590616823137),
 (('buddy', 'kibble'), 150.9126698530153),
 (('busy', 'buddy'), 150.9126698530153),
 (('kibble', 'nibble'), 150.9126698530153),
 (('the', 'purrfectly'), 133.76403744186524),
 (('co2', 'reactor'), 114.65088378856588),
 (('the', 'ratatouille'), 110.79827091439196),
 (('fluval', '305'), 109.3099619977161),
 (('seal', 'point'), 89.95369352475593),
 (('vacuum', 'cleaner'), 89.95369352475593),
 (('chocolate', 'labrador'), 83.92537094465293),
 (('canister', 'vacuum'), 77.73640748365806),
 (('polaris', 's4212'), 70.88932086507265),
 (('s4212', 'canister'), 68.20927405356312),
 (('rachel', 'ray'), 67.62365386978438),
 (('skinneeez', 'fox'), 67.62365386978438),
 (('halo', "'s"), 65.92581610326553),
 (('a', 'chocolate'), 61.68654079046243),
 (('a', 'seal'), 61.68654079046243),
 (('the', 'busy'), 56.80779789186

In [118]:
finder2.score_ngrams(bigram_measures.pmi)

[(('rachel', 'ray'), 8.315602456637947),
 (('skinneeez', 'fox'), 8.315602456637947),
 (('seal', 'point'), 7.830175629467706),
 (('vacuum', 'cleaner'), 7.830175629467706),
 (('chocolate', 'labrador'), 7.637530551525311),
 (('canister', 'vacuum'), 7.315602456637948),
 (('co2', 'reactor'), 7.1780989328880125),
 (('polaris', 's4212'), 7.10503547069829),
 (('s4212', 'canister'), 6.95303237725324),
 (('buddy', 'kibble'), 6.937090833384217),
 (('busy', 'buddy'), 6.937090833384217),
 (('kibble', 'nibble'), 6.937090833384217),
 (('fluval', '305'), 6.550067710274971),
 (('miele', 'does'), 6.4676055500829985),
 (('a', 'chocolate'), 6.113968595468298),
 (('a', 'seal'), 6.113968595468298),
 (('frenzy', 'refill'), 5.993674361750585),
 (('ratatouille', 'frenzy'), 5.993674361750585),
 (('liquid', 'ubiquinol'), 5.830175629467706),
 (('bissell', 'vac'), 5.637530551525311),
 (('halo', "'s"), 5.637530551525311),
 (('ubiquinol', 'coq10'), 5.598850083361251),
 (('purrfectly', 'chicken'), 5.593136432166856),

In [119]:
finder2.score_ngrams(bigram_measures.dice)

[(('buddy', 'kibble'), 1.0),
 (('busy', 'buddy'), 1.0),
 (('frenzy', 'refill'), 1.0),
 (('kibble', 'nibble'), 1.0),
 (('rachel', 'ray'), 1.0),
 (('ratatouille', 'frenzy'), 1.0),
 (('seal', 'point'), 1.0),
 (('skinneeez', 'fox'), 1.0),
 (('vacuum', 'cleaner'), 1.0),
 (('co2', 'reactor'), 0.9523809523809523),
 (('purrfectly', 'chicken'), 0.9523809523809523),
 (('chocolate', 'labrador'), 0.9333333333333333),
 (('ubiquinol', 'coq10'), 0.8363636363636363),
 (('canister', 'vacuum'), 0.8235294117647058),
 (('fluval', '305'), 0.7857142857142857),
 (('polaris', 's4212'), 0.7777777777777778),
 (('s4212', 'canister'), 0.7368421052631579),
 (('halo', "'s"), 0.5),
 (('miele', 'does'), 0.5),
 (('a', 'chocolate'), 0.4666666666666667),
 (('a', 'seal'), 0.4666666666666667),
 (('catit', 'to'), 0.4090909090909091),
 (('bissell', 'vac'), 0.3157894736842105),
 (('.', 'halo'), 0.3050847457627119),
 (('liquid', 'ubiquinol'), 0.30303030303030304),
 (('the', 'purrfectly'), 0.27906976744186046),
 (('because', '

Я считаю, ранжирование по Dice's coefficient лучше всего отбирает нужные нам коллокации

In [120]:
collocations = [x[0] for x in finder2.score_ngrams(bigram_measures.dice)] # нам не нужны скоры

In [121]:
collocations[:3]

[('buddy', 'kibble'), ('busy', 'buddy'), ('frenzy', 'refill')]

Группируем коллокации:

In [122]:
groupings = {}
for c in collocations:
  value = c[0] + ' ' + c[1]
  if c[0] in groupings.keys() and value not in groupings[c[0]]:
    groupings[c[0]].append(value)
  else:
    groupings[c[0]] = []
    groupings[c[0]].append(value)
  if c[1] in groupings.keys() and value not in groupings[c[1]]:
    groupings[c[1]].append(value)
  else:
    groupings[c[1]] = []
    groupings[c[1]].append(value)

Количество получившихся категорий:

In [123]:
len(groupings.keys())

46

In [125]:
groupings.keys()

dict_keys(['buddy', 'kibble', 'busy', 'frenzy', 'refill', 'nibble', 'rachel', 'ray', 'ratatouille', 'seal', 'point', 'skinneeez', 'fox', 'vacuum', 'cleaner', 'co2', 'reactor', 'purrfectly', 'chicken', 'chocolate', 'labrador', 'ubiquinol', 'coq10', 'canister', 'fluval', '305', 'polaris', 's4212', 'halo', "'s", 'miele', 'does', 'a', 'catit', 'to', 'bissell', 'vac', '.', 'liquid', 'the', 'because', 'of', 'ovo', 'habitrail', 'this', 'in'])

Теперь можем вывести, например, категорию сухого корма для животных:

In [126]:
groupings['kibble']

['buddy kibble', 'kibble nibble']

Поскольку мы урезали датасет для ускорения работы станзы, коллокаций по каждой категории получилось не так много.

## 5.

In [127]:
groupings['vacuum']

['vacuum cleaner', 'canister vacuum']

In [128]:
groupings['cleaner']

['vacuum cleaner']

In [129]:
groupings['chicken']

['purrfectly chicken']

In [130]:
groupings['ubiquinol']

['ubiquinol coq10', 'liquid ubiquinol']

In [131]:
groupings['purrfectly']

['purrfectly chicken', 'the purrfectly']